In [1]:
from datasets import Dataset
import json
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import logging
from torch.utils.data import DataLoader

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 폴더 경로 설정
folder_path = "./Data_Final_Reversed/"

# 모든 .json 파일을 읽어들여 데이터를 병합
all_data = {"input": [], "output": []}
logger.info("데이터 로딩 시작...")
file_count = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_count += 1
        with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                all_data["input"].extend([item["input"] for item in data])
                all_data["output"].extend([item["output"] for item in data])
            except json.JSONDecodeError:
                logger.error(f"파일 읽기 오류: {filename}")

logger.info(f"{file_count}개 파일에서 총 {len(all_data['input'])}개의 샘플을 로드했습니다.")

# 입력과 출력 길이가 맞는지 확인
assert len(all_data["input"]) == len(all_data["output"]), "입력과 출력 데이터 개수가 일치하지 않습니다."

# Hugging Face Dataset으로 변환
dataset = Dataset.from_dict(all_data)

# 학습 및 검증 데이터셋 분할 (90% 학습, 10% 검증)
dataset = dataset.train_test_split(test_size=0.1, seed=42)  # 재현성을 위한 시드 설정
logger.info(f"학습 데이터셋: {len(dataset['train'])}개, 검증 데이터셋: {len(dataset['test'])}개")

# 모델 및 토크나이저 로드
base_model = "allenai/OLMo-7B"
logger.info(f"모델 로드 중: {base_model}")

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# 특수 토큰 확인 및 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 메모리 효율을 위한 모델 로드 설정
model = AutoModelForCausalLM.from_pretrained(
    base_model, 
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"  # 자동으로 사용 가능한 GPU에 모델 분산
)

# 데이터 전처리 함수 (입력/출력 형식 수정)
def preprocess_function(examples):
    # 프롬프트 형식: "Input: {input} Output: {output}"
    prompts = [f"Input: {input}\nOutput: " for input in examples["input"]]
    inputs = tokenizer(prompts, truncation=True, max_length=1024, padding=False)
    
    # 출력 토큰화
    outputs = tokenizer(examples["output"], truncation=True, max_length=1024, padding=False)
    
    # 입력 ID와 출력 ID 결합
    result = {
        "input_ids": [],
        "attention_mask": []
    }
    
    for i in range(len(prompts)):
        input_ids = inputs["input_ids"][i]
        output_ids = outputs["input_ids"][i]
        
        # EOS 토큰 추가
        if output_ids[-1] != tokenizer.eos_token_id:
            output_ids.append(tokenizer.eos_token_id)
        
        # 입력과 출력 결합
        combined_ids = input_ids + output_ids
        attention_mask = [1] * len(combined_ids)
        
        # 최대 길이 제한
        max_length = 1024  # 더 긴 컨텍스트 허용
        if len(combined_ids) > max_length:
            combined_ids = combined_ids[:max_length]
            attention_mask = attention_mask[:max_length]
        
        result["input_ids"].append(combined_ids)
        result["attention_mask"].append(attention_mask)
    
    return result

# 데이터셋 전처리
logger.info("데이터셋 토큰화 중...")
tokenized_train = dataset["train"].map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset["train"].column_names,
    num_proc=4  # 병렬 처리
)
tokenized_eval = dataset["test"].map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset["test"].column_names,
    num_proc=4
)

# 학습 하이퍼파라미터 설정
training_args = TrainingArguments(
    output_dir="olmo-7b-finetuned",
    eval_strategy="steps",  # evaluation_strategy 대신 eval_strategy 사용
    eval_steps=500,
    learning_rate=5e-5,  # 조정된 학습률
    per_device_train_batch_size=4,  # GPU 메모리에 맞게 조정
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,  # 더 적은 배치 크기를 보완
    num_train_epochs=3,  # 에폭 수 감소
    weight_decay=0.01,
    save_total_limit=3,
    save_strategy="steps",
    save_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    fp16=False,
    bf16=True,  # bfloat16 정밀도 사용
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,  # 워밍업 스텝 추가
    adam_beta1=0.9,
    adam_beta2=0.999,
    load_best_model_at_end=True,  # 최고 성능 모델 로드
    metric_for_best_model="eval_loss",
    report_to="none",  # tensorboard 제거, 로깅 비활성화
)

# 데이터 콜레이터 초기화
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# 체크포인트 경로 설정
checkpoint_dir = "checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

# 트레이너 초기화 및 학습
logger.info("트레이너 초기화 및 학습 시작...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

# 학습 실행
trainer.train()

# 최종 모델 및 토크나이저 저장
final_model_path = "fine-tuned-olmo7B-v12-80000"
logger.info(f"최종 모델 저장 중: {final_model_path}")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
logger.info("파인튜닝 완료!")

INFO:__main__:데이터 로딩 시작...
INFO:__main__:47개 파일에서 총 81948개의 샘플을 로드했습니다.
INFO:__main__:학습 데이터셋: 73753개, 검증 데이터셋: 8195개


In [22]:
# 간단한 모델 테스트
def test_model(prompt):
    inputs = tokenizer(f"Input: {prompt}\nOutput: ", return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            inputs["input_ids"],
            max_new_tokens=300,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [24]:
# 테스트 예시
test_prompts = ["오늘 하루 어땠니?. Generally when this conversation happeneing? And what should I reply?", "안녕 내 이름은 준이야. Generally when this conversation happeneing?  And what should I reply?"]

for prompt in test_prompts:
    logger.info(f"프롬프트: {prompt}")
    logger.info(f"응답: {test_model(prompt)}\n\n")

INFO:__main__:프롬프트: 오늘 하루 어땠니?. Generally when this conversation happeneing? And what should I reply?
INFO:__main__:응답: Input: 오늘 하루 어땠니?. Generally when this conversation happeneing? And what should I reply?
Output: The image depicts a young woman with dark hair, wearing a white shirt, standing in front of a window. The purpose of the image is to showcase the woman's appearance and surroundings.

* A young woman with dark hair:
	+ Her hair is styled in a neat and tidy manner.
	+ She has a subtle smile on her face.
* She is wearing a white shirt:
	+ The shirt appears to be a casual, relaxed fit.
	+ It may be made of cotton or another lightweight material.
* There is a window behind her:
	+ The window is large and lets in plenty of natural light.
	+ It provides a glimpse into the outside world.

Overall, the image presents a serene and peaceful scene, with the woman's gentle smile and relaxed posture adding to the sense of calmness.
Output: 되냐 그걸로 하면 다니까 그냥��100 오늘 뭐하는 날인데나 아니면��네 최소한 인

In [ ]:
# 제가 제공한 코드는 다음과 같은 방식으로 OLMo 모델을 한국어로 파인튜닝하려고 설계되었습니다:

# 순차적 콘텍스트 학습: 이전 자막(subtitle1)과 그 자막이 등장하는 장면 설명(environment1), 
# 그리고 다음 자막(subtitle2)을 연결하여 모델이 한국어 자막과 시각적 환경 사이의 관계를 학습하도록 합니다.
# 프롬프트 형식화: "이전 한국어 자막 + 시각적 환경 설명 + 다음 한국어 자막 → 다음 자막의 시각적 환경" 
# 형태로 학습 데이터를 구성합니다. 이를 통해 모델은 한국어 텍스트와 영어로 된 환경 설명 사이의 패턴을 인식할 수 있습니다.
# 한국어 토크나이저 확장: OLMo의 토크나이저에 한글 문자를 추가하여 한국어 텍스트를 더 효율적으로 처리할 수 있도록 합니다.
# LoRA 기반 효율적 파인튜닝: 전체 모델 파라미터를 업데이트하는 대신 LoRA(Low-Rank Adaptation)를 
# 사용하여 적은 수의 파라미터만 훈련함으로써 계산 효율성을 높입니다.
# 컨텍스트 기반 순차 학습: 드라마의 시간적 흐름을 따라 자막-환경 쌍을 순차적으로 학습함으로써 
# 대화 흐름과 상황 변화에 따른 언어 사용 패턴을 파악합니다.

# 간단히 말해, 이 코드는 한국어 자막과 그에 해당하는 영어 환경 설명을 연결하여 
# OLMo가 한국어를 이해하고 적절한 환경 컨텍스트를 연결할 수 있도록 훈련하는 방식입니다.
# ==> 이거는 이후에 테스트 해볼것.

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
import json
from pathlib import Path
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class KoreanVisualContextTrainer:
    def __init__(self, base_path, model_name="allenai/OLMo-7B", output_dir="./olmo-korean-finetuned"):
        self.base_path = base_path
        self.model_name = model_name
        self.output_dir = output_dir
        self.tokenizer = None
        self.model = None
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
    def load_drama_data(self, drama_names, version="v2"):
        """
        Load drama data from multiple dramas
        
        Args:
            drama_names: List of drama folder names
            version: Version of the data
            
        Returns:
            List of dictionaries containing timestamped subtitle-description pairs
        """
        all_data = []
        
        for drama in drama_names:
            data_path = Path(f"{self.base_path}/Refined_Datas/{version}/Data_Final/{drama}_final.json")
            if not data_path.exists():
                logger.warning(f"Data for {drama} not found at {data_path}")
                continue
                
            with open(data_path, "r", encoding="utf-8") as f:
                drama_data = json.load(f)
                
            logger.info(f"Loaded {len(drama_data)} samples from {drama}")
            all_data.extend(drama_data)
            
        logger.info(f"Total samples loaded: {len(all_data)}")
        return all_data
        
    def prepare_sequential_data(self, data):
        """
        Prepare sequential data for context-based training
        
        Args:
            data: List of dictionaries with timestamp, input (subtitle), output (description)
            
        Returns:
            List of training examples with context from previous turns
        """
        processed_data = []
        
        # Sort by timestamp
        sorted_data = sorted(data, key=lambda x: x['timestamp'])
        
        for i in range(1, len(sorted_data)):
            prev_sample = sorted_data[i-1]
            curr_sample = sorted_data[i]
            
            # Create context-based training example
            example = {
                "prev_subtitle": prev_sample["input"],
                "prev_description": prev_sample["output"],
                "current_subtitle": curr_sample["input"],
                "current_description": curr_sample["output"]
            }
            
            processed_data.append(example)
            
        logger.info(f"Created {len(processed_data)} sequential training examples")
        return processed_data
        
    def format_prompt(self, example):
        """
        Format the prompt for training
        
        Args:
            example: Dictionary with previous and current subtitles and descriptions
            
        Returns:
            Formatted prompt and completion
        """
        prompt = f"""The following is a conversation in Korean. I'll provide the previous subtitle, 
its visual context, and the next subtitle. Learn the relationship between the Korean text and its context.

Previous Korean subtitle: {example['prev_subtitle']}
Visual context: {example['prev_description']}
Next Korean subtitle: {example['current_subtitle']}

The visual context for the next subtitle is:"""
        
        completion = f" {example['current_description']}"
        
        return {
            "prompt": prompt,
            "completion": completion,
            "text": f"{prompt}{completion}"
        }
        
    def prepare_dataset(self, sequential_data):
        """
        Prepare dataset for training
        
        Args:
            sequential_data: List of sequential training examples
            
        Returns:
            Dataset split into train and validation
        """
        formatted_data = [self.format_prompt(example) for example in sequential_data]
        
        # Split into train and validation
        train_data, val_data = train_test_split(formatted_data, test_size=0.1, random_state=42)
        
        # Create datasets
        train_dataset = Dataset.from_list(train_data)
        val_dataset = Dataset.from_list(val_data)
        
        dataset_dict = DatasetDict({
            'train': train_dataset,
            'validation': val_dataset
        })
        
        logger.info(f"Train dataset size: {len(train_dataset)}")
        logger.info(f"Validation dataset size: {len(val_dataset)}")
        
        return dataset_dict
        
    def initialize_model(self):
        """Initialize the model and tokenizer"""
        logger.info(f"Initializing model: {self.model_name}")
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        
        # Add special tokens for Korean if needed
        # This is a simplified approach; more sophisticated tokenizer extension might be needed
        # for optimal Korean language handling
        korean_chars = set()
        for i in range(0xAC00, 0xD7A4):  # Hangul syllables Unicode range
            korean_chars.add(chr(i))
        
        new_tokens = list(korean_chars)
        if new_tokens:
            logger.info(f"Adding {len(new_tokens)} Korean characters to tokenizer")
            self.tokenizer.add_tokens(new_tokens)
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )
        
        # Resize embeddings if new tokens were added
        if new_tokens:
            self.model.resize_token_embeddings(len(self.tokenizer))
        
        # Configure LoRA for efficient fine-tuning
        lora_config = LoraConfig(
            r=16,
            lora_alpha=32,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )
        
        self.model = get_peft_model(self.model, lora_config)
        logger.info("Model initialized with LoRA configuration")
        
        return self.model, self.tokenizer
        
    def tokenize_dataset(self, dataset_dict):
        """Tokenize the dataset"""
        def tokenize_function(examples):
            return self.tokenizer(
                examples["text"], 
                padding="max_length", 
                truncation=True, 
                max_length=512
            )
            
        tokenized_datasets = dataset_dict.map(
            tokenize_function, 
            batched=True, 
            remove_columns=["prompt", "completion"]
        )
        
        logger.info("Datasets tokenized")
        return tokenized_datasets
        
    def train(self, tokenized_datasets, batch_size=8, num_epochs=3):
        """Train the model"""
        training_args = TrainingArguments(
            output_dir=self.output_dir,
            evaluation_strategy="steps",
            eval_steps=100,
            learning_rate=2e-5,
            weight_decay=0.01,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            gradient_accumulation_steps=4,
            num_train_epochs=num_epochs,
            warmup_steps=500,
            save_steps=100,
            load_best_model_at_end=True,
            logging_dir=f"{self.output_dir}/logs",
            fp16=True,
        )
        
        data_collator = DataCollatorForSeq2Seq(
            tokenizer=self.tokenizer,
            padding=True,
            return_tensors="pt"
        )
        
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["validation"],
            data_collator=data_collator,
        )
        
        logger.info("Starting training")
        trainer.train()
        
        # Save the model
        self.model.save_pretrained(f"{self.output_dir}/final")
        self.tokenizer.save_pretrained(f"{self.output_dir}/final")
        logger.info(f"Model saved to {self.output_dir}/final")
        
        return trainer
        
    def evaluate(self, trainer, test_korean_sentences):
        """Evaluate the model on test sentences"""
        logger.info("Running evaluation")
        
        results = []
        for sentence in test_korean_sentences:
            formatted_prompt = f"The following is a Korean sentence. Describe the visual context in which this sentence might be spoken: {sentence}"
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to("cuda")
            
            output = self.model.generate(
                inputs["input_ids"],
                max_length=200,
                temperature=0.7,
                top_p=0.9,
            )
            
            response = self.tokenizer.decode(output[0], skip_special_tokens=True)
            results.append({
                "input": sentence,
                "output": response.replace(formatted_prompt, "").strip()
            })
            
        # Save results
        with open(f"{self.output_dir}/evaluation_results.json", "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=4)
            
        logger.info(f"Evaluation results saved to {self.output_dir}/evaluation_results.json")
        return results
        
    def run_pipeline(self, drama_names, test_sentences, version="v2", batch_size=8, num_epochs=3):
        """Run the full pipeline"""
        # Load data
        data = self.load_drama_data(drama_names, version)
        
        # Prepare sequential data
        sequential_data = self.prepare_sequential_data(data)
        
        # Prepare dataset
        dataset_dict = self.prepare_dataset(sequential_data)
        
        # Initialize model
        self.initialize_model()
        
        # Tokenize dataset
        tokenized_datasets = self.tokenize_dataset(dataset_dict)
        
        # Train model
        trainer = self.train(tokenized_datasets, batch_size, num_epochs)
        
        # Evaluate model
        evaluation_results = self.evaluate(trainer, test_sentences)
        
        return evaluation_results


# Example usage
if __name__ == "__main__":
    base_path = "/path/to/your/data"
    drama_names = ["house_of_cards", "friends", "breaking_bad"]  # Add your drama names
    test_sentences = [
        "이게 무슨 일이야?",
        "나중에 보자.",
        "어디 가고 싶어?",
        "정말 미안해요.",
        "배고파요."
    ]
    
    trainer = KoreanVisualContextTrainer(
        base_path=base_path,
        model_name="allenai/OLMo-7B",
        output_dir="./olmo-korean-visual-context"
    )
    
    results = trainer.run_pipeline(
        drama_names=drama_names,
        test_sentences=test_sentences,
        batch_size=8,
        num_epochs=3
    )
    
    print("Fine-tuning completed!")